# **머신러닝을 활용한 전처리**
규격화된 일정량의 데이터를 활용하여 많은 데이터 처리용 알고리즘
- https://github.com/lovit/soynlp 

# **1 WPM (Word Piece Model)**
- **[BPE(Byte Pair Encoding)](https://wikidocs.net/22592)** 알고리즘 (1994) Philip Gage 논문이 배경
- **WPM** 모델링 기법 : 2016 년 **Pre Processing 모델링을** 활용한 Stemming 기법

## **01 defaultDict 을 활용한 단어조합기**
**[defaultDict()](https://itholic.github.io/python-defaultdict/)** 을 활용한 **N-Gram** 생성 후 **단어 조합하기** 연
습
1. 식별이 가능한 Token 단위로 학습데이터 만들기

In [1]:
# 단어 Token 을 음절단위로 구분하기
" ".join(list("low")+["</w>"]) 

'l o w </w>'

In [2]:
# Token 과 빈도수를 활용한 
data = {
    " l o w </w>":5,
    " l o w e s t </w>":2,
    " n e w e r </w>":6,
    " w i d e r </w>":3,
}

from collections import defaultdict
pair      = defaultdict(int)
candidate = {}
n_gram    = 2

for k, v in data.items():
    token = k.split()
    for i in range(len(token)-1):
        pair[tuple(token[ i : i+n_gram ])] += v
pair

defaultdict(int,
            {('l', 'o'): 7,
             ('o', 'w'): 7,
             ('w', '</w>'): 5,
             ('w', 'e'): 8,
             ('e', 's'): 2,
             ('s', 't'): 2,
             ('t', '</w>'): 2,
             ('n', 'e'): 6,
             ('e', 'w'): 6,
             ('e', 'r'): 9,
             ('r', '</w>'): 9,
             ('w', 'i'): 3,
             ('i', 'd'): 3,
             ('d', 'e'): 3})

In [3]:
# .get : Tuple 의 객체들을 따로 따로 묶어서 판단 합니다.
maxKey = max(pair, key=pair.get) # pair.get : <function defaultdict.get>
maxKey

('e', 'r')

In [5]:
# 함께 출현하는 빈도가 높을수록 단어로 묶일 확률이 높다
# 작업결과 "er" 이 묶인걸 볼 수 있습니다.
newData = {}
import re
for k, v in data.items():
    newKey = re.sub(" ".join(maxKey), "".join(maxKey), k)
    newData[newKey] = v
newData

{' l o w </w>': 5,
 ' l o w e s t </w>': 2,
 ' n e w er </w>': 6,
 ' w i d er </w>': 3}

## **2 Word Piece Model 한글실습**
- defaultDict 단어 token 을 만든 뒤,
- Bi-Gram 빈도를 기준으로 단어 Token 을 찾기

In [6]:
# Ngram defaultDict 만들기
def findNgram(data):
    pair = defaultdict(int)
    candidate = {}
    for k, v in data.items():
        token = k.split()
        for i in range(len(token)-1):
            pair[tuple(token[ i : i+2 ])] += v
    return pair

# MaxKey 빈도값을 활용하여 단어 생성하기
def mergeNgram(maxKey, data):
    newData = {}
    for k, v in data.items():
        newKey = re.sub(" ".join(maxKey), "".join(maxKey), k)
        newData[newKey] = v
    return newData

In [7]:
# 어절 Token 목록을 활용한 단어 찾기
data = {
    " 아 버 지 께 </w>":5,
    " 아 버 지 를 </w>":2,
    " 아 버 지 에 게 </w>":6,
    " 아 버 지 와 </w>":3,
}

for _ in range(2):
    pairList = findNgram(data)
    maxKey = max(pairList, key=pairList.get)
    data = mergeNgram(maxKey, data)
data

{' 아버지 께 </w>': 5, ' 아버지 를 </w>': 2, ' 아버지 에 게 </w>': 6, ' 아버지 와 </w>': 3}

## **3 Word Piece Model 한글실습 2**
- 반복 횟수인 MaxKey 값을 자동으로 찾기

In [8]:
data = {
    "아 버 지 께 </w>":5,
    "아 버 지 를 </w>":2,
    "아 버 지 에 게 </w>":6,
    "아 버 지 와 </w>":3,
}
data.values()

dict_values([5, 2, 6, 3])

In [9]:
from collections import defaultdict
def findNagram(data):
    pair = defaultdict(int)
    for k, v in data.items():
        tokens = k.split()
        for i in range(len(tokens)-1):
            pair[tuple(tokens[i:i+2])] += v
    return pair

# maxValue 변수를 추가해 반복 횟수를 제한 합니다
def mergeNgram(maxKey, data):
    newData = dict()
    for k, v in data.items():
        newKey = re.sub(" ".join(maxKey),
                        "".join(maxKey), k)
        newData[newKey] = v
    return newData

maxValue = max(data.values())
maxValue

6

In [10]:
for _ in range(1000):
    pairList = findNagram(data)
    maxKey   = max(pairList, key=pairList.get)
    print(maxKey, pairList[maxKey])
    
    if pairList[maxKey] > maxValue:
        data = mergeNgram(maxKey, data)
    else: 
        break
data

('아', '버') 16
('아버', '지') 16
('아버지', '에') 6


{'아버지 께 </w>': 5, '아버지 를 </w>': 2, '아버지 에 게 </w>': 6, '아버지 와 </w>': 3}

In [11]:
pairList

defaultdict(int,
            {('아버지', '께'): 5,
             ('께', '</w>'): 5,
             ('아버지', '를'): 2,
             ('를', '</w>'): 2,
             ('아버지', '에'): 6,
             ('에', '게'): 6,
             ('게', '</w>'): 6,
             ('아버지', '와'): 3,
             ('와', '</w>'): 3})

<br/>

# **2 레시피 데이터 적용하기**
- **[BPE(Byte Pair Encoding)](https://wikidocs.net/22592)** 알고리즘 (1994) Philip Gage 논문이 배경
- **WPM** 모델링 기법 : 2016 년 **Pre Processing 모델링을** 활용한 Stemming 기법

## **01 defaultDict 을 활용한 단어조합기**
**[defaultDict()](https://itholic.github.io/python-defaultdict/)** 을 활용한 **N-Gram** 생성 후 **단어 조합하기** 연
습
1. [f-string 사용법](https://bluese05.tistory.com/70
1. 식별이 가능한 Token 단위로 학습데이터 만들기

In [1]:
# 출처: https://bluese05.tistory.com/70 [ㅍㅍㅋㄷ]
tuple = ('Hi, I am', 'song', 123)
f'tuple: {tuple}' #>>> "tuple: ('Hi, I am', 'song', 123)"

"tuple: ('Hi, I am', 'song', 123)"

In [5]:
# r'' 은 백슬래시를 함수처리서 제외 함
print(r"머신러닝 \n 알고리즘")
print("머신러닝 \n 알고리즘")

In [ ]:
key = '' 

In [18]:
f'{a} Python {b} tensorflow'#.(a=3.6, b=2)

NameError: name 'a' is not defined

In [16]:
def greet(k,v):
    return f"hello {k} in work {v}"

In [ ]:
units = {
    "15ml":["1큰술","1T","1Ts","3t"],
    "5ml":["1작은술","1t","1ts"],    
    "200ml":["1컵","1Cup","1C"],
    "250ml":["16T", "1C"],
    "180ml":["1종이컵"],
    "28.3g":["1oz"],
    "453g":["1파운드","lb"],
    "3780ml":["1갤런","gallon"],
    "2g":["1꼬집",'약간'],
    "4g":["조금"],
    "6g":["조금"],
    "10g":["적당량"],
    "50g":["1줌"], # 나물
    "100g":["1큰줌"], # 나물
    "13마리":["1줌"], # 멸치
    "26마리":["1큰줌"], # 멸치
    "100g":["1주먹"], # 여자 어른의 주먹크기
    "3cm":["1토막"],
    "1알":["1톨","1쪽"], # 마늘, 생강 등
    "600g":["1근"], # 고기
    "400g":["1근"], # 채소
    "200g":["1봉지"],
}

unit_tokens = []
for _ in [v  for k,v in units.items()]:
    unit_tokens += _
", ".join(sorted(set(unit_tokens)))

# **식약처 레시피로 이름구분 및 단위 기준 전처리**
레시피 데이터 전처리 및 단위기준 일치사전 만들기
```
계량법 안내
1큰술(1T, 1Ts) = 1숟가락 	15ml = 3t (밥숟가락 뜨면 1큰술)
1작은술(1t, 1ts)              5ml (티스푼으로는 2스푼이 1작은술)
1컵(1Cup, 1C) 	200ml = 16T (한국,중국,일본)  (서양(미국)은 1C가 240~250ml)
1종이컵 	180ml; 1oz 	28.3g
1파운드(lb) 	약 0.453 킬로그램(kg)
1갤런(gallon) 	약 3.78 리터(ℓ)
1꼬집 	약 2g 정도이며 '약간'이라고 표현하기도 함
조금   	약간의 2~3배
적당량 	기호에 따라 마음대로 조절해서 넣으란 표현
1줌 	    한손 가득 (예시 : 멸치 1줌 = 국멸치인 경우 12~15마리, 나물 1줌은 50g) 크게 1줌 = 2줌 [1줌의 두배]
1주먹 	여자 어른의 주먹크기, 고기로는 100g
1토막 	2~3cm두께 정도의 분량
마늘 1톨 	깐 마늘 한쪽
생강 1쪽 	마늘 1톨의 크기와 비슷
생강 1톨 	아기 손바닥만한 크기의 통생강 1개
고기 1근 	600g
채소 1근 	400g
채소 1봉지 	200g 정도
```